# 法律語法形式化
- version: 20250314
- spec version: 3.1
- history:
    - 20250312: 讀寫 json, 基本支援 LLM 建構 json
    - 20250313: 用 code 產生小架構的法規，增加管理功能
    - 20250314: 加入 query 的一些功能，順便測試一下資料
    - 20250319: 改善 prompt, 生成更正確，加幾個法

- env: 2504
## 起始化-物件與functions

In [11]:
import json
from enum import Enum
import re
import os

dir_txt = "txt"
dir_json = "json"
def get_law_names_from_directory(directory_path):
    """
    從目錄中取得所有法規名稱的列表
    :param directory_path: 目錄的路徑
    :return: 法規名稱的列表
    """
    law_names = set()
    # Check if directory exists
    if not os.path.exists(directory_path):
        print(f"Directory not found: {directory_path}")
        return list(law_names)
    for filename in os.listdir(directory_path):
        if filename.endswith("_law_regulation.json"):
            law_name = filename.split("_law_regulation")[0]
            law_names.add(law_name)
    return list(law_names)
def handle_regex(regex,file_path,type="col2"): 
    """
    return lines
    """
    print(f"parse file_path:{file_path}")
    with open(file_path, 'r', encoding='utf-8') as file:
            test_str = file.read()
    #print(test_str)
    matches = re.finditer(regex, test_str, re.DOTALL) #re.MULTILINE
    lines = []
    if type=="col2": #regex="\*\*Q：\*\*(.*)\n\*\*A：\*\*(.*)\n"
        for matchNum, match in enumerate(matches, start=1):
            for groupNum in range(0, len(match.groups())):
                groupNum = groupNum + 1
                mark = "Q" if groupNum==1 else "A"
                group = match.group(groupNum).replace("*","")
                print_str = f"{mark}:{group}"
                print(print_str ) 
                lines.append(print_str)
    if type=="col1":
        
        for matchNum, match in enumerate(matches, start=1):
            
            for groupNum in range(0, len(match.groups())):
                groupNum = groupNum + 1
                group = match.group(groupNum)
                print_str = f"{group}"
                #print(print_str ) 
                lines.append(print_str)
    return lines    

# Helper function: Load JSON data from file
def load_json_data(filepath):
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            return json.load(f)
    except FileNotFoundError:
        print(f"File not found: {filepath}")
        return None
    except json.JSONDecodeError:
        print(f"JSON format error: {filepath}")
        return None
    
class ConceptCategory(Enum):
    """Legal concept categories enumeration."""
    CORE_CONCEPT_DEFINITION = "核心概念 - 定義" # You can keep Chinese values for display if needed
    # ... 其他類別 (other categories)

class LawMetadata:
    """
    Law MetaData object, used to integrate different types of law information.
    """
    def __init__(self, law_name = None, law_regulation=None, legal_concepts=None, hierarchy_relations=None, law_relations=None, law_articles=None):
        """
        Initializes LawMetaData object.

        Args:
            law_regulation (dict, optional): Law regulation metadata, expected as a single dict. Defaults to None.
            legal_concepts (list of dict, optional): Legal concepts metadata, expected as a list of dict. Defaults to None.
            hierarchy_relations (list of dict, optional): Hierarchy relations metadata, expected as a list of dict. Defaults to None.
            law_relations (list of dict, optional): Law relations metadata, expected as a list of dict. Defaults to None.
            law_articles (list of dict, optional): Law articles metadata, expected as a list of dict. Defaults to None.
        """
        self.law_name = law_name or "NA"
        self.law_regulation = law_regulation or {}
        self.legal_concepts = legal_concepts or []
        self.hierarchy_relations = hierarchy_relations or []
        self.law_relations = law_relations or []
        self.law_articles = law_articles or []
        self.law_name = self.law_regulation.get("法規名稱", "default_law") # Default prefix if not found
        self.short_name = self.law_name


    @classmethod
    def from_json_files(cls, law_regulation, legal_concepts, hierarchy_relations, law_relations, law_articles):
        """
        Reads law MetaData from separate JSON files.

        Args:
            law_regulation (str): Filepath for law regulation metadata JSON file.
            legal_concepts (str): Filepath for legal concepts metadata JSON file.
            hierarchy_relations (str): Filepath for hierarchy relations metadata JSON file.
            law_relations (str): Filepath for law relations metadata JSON file.
            law_articles (str): Filepath for law articles metadata JSON file.

        Returns:
            LawMetaData: LawMetaData object, or None if any file reading fails.
        """
        law_regulation_data = load_json_data(law_regulation)
        legal_concepts_data = load_json_data(legal_concepts)
        hierarchy_relations_data = load_json_data(hierarchy_relations)
        law_relations_data = load_json_data(law_relations)
        law_articles_data = load_json_data(law_articles)

        # Check if all essential components are loaded
        if not all([law_regulation_data, law_articles_data]): # Core components
            print(f"Error: Core data (regulation or articles) missing for one of the paths provided.")
            print(f"Paths: LR:{law_regulation}, LC:{legal_concepts}, LH:{hierarchy_relations}, LRel:{law_relations}, LA:{law_articles}")
            return None
        # Non-critical components can be empty lists if files are not found but are specified
        if legal_concepts_data is None: legal_concepts_data = []
        if hierarchy_relations_data is None: hierarchy_relations_data = []
        if law_relations_data is None: law_relations_data = []

        return cls(
            law_regulation=law_regulation_data,
            legal_concepts=legal_concepts_data,
            hierarchy_relations=hierarchy_relations_data,
            law_relations=law_relations_data,
            law_articles=law_articles_data
        )
    

    def to_json_files(self, output_prefix="gpa"):
        """
        Exports LawMetaData object to separate JSON files with law name prefix.

        Args:
            output_prefix (str, optional): Prefix for output filenames. Defaults to "gpa".
        """
        if not output_prefix:
            output_prefix = self.law_name # Fallback to law prefix if output_prefix is not provided

        filepaths = {
            "law_regulation": f"{output_prefix}_law_regulation.json",
            "legal_concepts": f"{output_prefix}_legal_concepts.json",
            "hierarchy_relations": f"{output_prefix}_hierarchy_relations.json",
            "law_relations": f"{output_prefix}_law_relations.json",
            "law_articles": f"{output_prefix}_law_articles.json"
        }

        data_to_export = {
            "law_regulation": self.law_regulation,
            "legal_concepts": [
                {**concept, "概念類別": concept["概念類別"].value if isinstance(concept.get("概念類別"), ConceptCategory) else concept.get("概念類別")}
                for concept in self.legal_concepts
            ],
            "hierarchy_relations": self.hierarchy_relations,
            "law_relations": self.law_relations,
            "law_articles": self.law_articles
        }

        for key, filepath in filepaths.items():
            # Ensure directory for filepath exists
            os.makedirs(os.path.dirname(filepath), exist_ok=True)
            with open(filepath, 'w', encoding='utf-8') as f:
                json.dump(data_to_export[key], f, indent=2, ensure_ascii=False)
        print(f"Exported all components for '{self.law_name}' with prefix '{output_prefix}'.")

    def renew_id(self):
        law_id = f"LT_{self.law_name}"
        self.law_regulation["代號"]=law_id
        for article in self.law_articles:
            article_code_current = article.get("代號", "")
            if not article_code_current.startswith("LA_"):
                article_id = f"LA_{self.law_name}_{article.get('條號', 'UnknownArticle')}"
                article["代號"]=article_id
        for concept in self.legal_concepts:
            concept_code_current = concept.get("代號", "")
            if not concept_code_current.startswith("LC_"):
                concept_id = f"LC_{self.law_name}_{concept.get('詞彙名稱', 'UnknownConcept')}"
                concept['代號']=concept_id
        seq = 1
        for relation in self.law_relations:
            relation_code_current = relation.get("代號", "")
            if not relation_code_current.startswith("LR_"):
                relation_id = f"LR_{self.law_name}_{seq}"
                relation['代號']=relation_id
            seq += 1
        seq_hier = 1 # Separate sequence for hierarchy if needed, or use a more descriptive ID generation
        for hierarchy in self.hierarchy_relations:
            hier_code_current = hierarchy.get("關係代號", "")
            if not hier_code_current.startswith("LH_"):
                # Using related law and type might be more stable if available and unique
                hierarchy_id = f"LH_{self.law_name}_{hierarchy.get('關聯法規', 'UnknownRelatedLaw')}_{seq_hier}"
                hierarchy['關係代號']=hierarchy_id
            seq_hier +=1
        print(f"IDs renewed for LawMetadata: {self.law_name}")


    def __repr__(self):
        return f"LawMetadata(law_name='{self.law_regulation.get('法規名稱', 'N/A')}', concept_count={len(self.legal_concepts)}, ...)"

class LawMetadataMgr:
    def __init__(self, db_conn=None):
        self.lms = {}
        self.short_names = {}
        self.dir_json = "json"
        self.db_conn = db_conn
        if self.db_conn:
            print("LawMetadataMgr initialized with a database connection.")
        else:
            print("LawMetadataMgr initialized without a database connection (JSON mode only).")

    def add_lm(self, lm , short_name=None):
        if not lm or not lm.law_name or not lm.law_regulation.get('法規名稱'):
            print("Error: Cannot add invalid or incomplete LawMetadata object.")
            return
        
        actual_short_name = short_name if short_name else lm.law_name
        lm.short_name = actual_short_name
        
        self.short_names[actual_short_name] = lm.law_name 
        self.lms[lm.law_name] = lm
        print(f"Added '{lm.law_name}' (short: '{actual_short_name}') to in-memory manager.")

        if self.db_conn:
            print(f"DB connection found. Attempting to upsert LawMetadata for '{lm.law_name}' to database.")
            try:
                # Ensure IDs are in the desired format before upserting if they come from JSON
                # lm.renew_id() # Uncomment if JSON data IDs are not guaranteed to be in final format
                success = upsert_law_metadata_to_db(lm, self.db_conn)
                if success:
                    print(f"Successfully upserted '{lm.law_name}' to database.")
                else:
                    print(f"Warning: Failed to upsert '{lm.law_name}' to database. It remains in memory only.")
            except Exception as e:
                print(f"Error during DB upsert for '{lm.law_name}': {e}. It remains in memory only.")

    def remove_lm(self, law_name_or_short_name):
        law_name_to_remove = self.short_names.get(law_name_or_short_name, law_name_or_short_name)
        
        if law_name_to_remove in self.lms:
            lm_to_remove = self.lms[law_name_to_remove]
            law_code_to_delete = lm_to_remove.law_regulation.get('代號')
            
            del self.lms[law_name_to_remove]
            if law_name_or_short_name in self.short_names and self.short_names[law_name_or_short_name] == law_name_to_remove:
                del self.short_names[law_name_or_short_name]
            print(f"Removed '{law_name_to_remove}' from in-memory manager.")

            if self.db_conn:
                if law_code_to_delete:
                    print(f"DB connection found. Attempting to delete LawMetadata for law code '{law_code_to_delete}' from database.")
                    try:
                        delete_law_metadata_from_db(law_code_to_delete, self.db_conn)
                    except Exception as e:
                        print(f"Error during DB delete for law code '{law_code_to_delete}': {e}.")
                else:
                    print(f"Warning: Cannot delete '{law_name_to_remove}' from DB as its code ('代號') is missing.")
        else:
            print(f"Law '{law_name_or_short_name}' not found in manager.")

    def find_lm(self, law_name_or_short_name):
        law_name_to_find = self.short_names.get(law_name_or_short_name, law_name_or_short_name)
        return self.lms.get(law_name_to_find, None)

    def load_lm_bynames_from_json(self, short_names_list):
        """Loads LawMetadata from JSON files based on a list of short names."""
        loaded_count = 0
        for short_name in short_names_list:
            # Construct full paths for JSON files
            base_path = os.path.join(self.dir_json, short_name)
            lm = LawMetadata.from_json_files(
                f"{base_path}_law_regulation.json",
                f"{base_path}_legal_concepts.json",
                f"{base_path}_hierarchy_relations.json",
                f"{base_path}_law_relations.json",
                f"{base_path}_law_articles.json"
            )
            if lm:
                # lm.renew_id() # Call renew_id here if IDs from JSON are not standardized for DB
                self.add_lm(lm, short_name)
                loaded_count += 1
        print(f"Finished loading from JSON. {loaded_count} laws processed.")
        
    def load_lm_from_db(self, law_code_to_load: str = None):
        """Loads LawMetadata from the database for a specific law code or all laws."""
        if not self.db_conn:
            print("Error: Database connection not available for load_lm_from_db.")
            return

        laws_query = "SELECT id, code, name, data FROM laws"
        params = []
        if law_code_to_load:
            laws_query += " WHERE code = %s"
            params.append(law_code_to_load)
        
        base_law_records = fetch_query(self.db_conn, laws_query, tuple(params) if params else None)
        if not base_law_records:
            print(f"No laws found in DB for code '{law_code_to_load if law_code_to_load else 'ALL'}'.")
            return

        loaded_count = 0
        # Clear existing LMs if loading all, to avoid duplicates if run multiple times
        if not law_code_to_load:
            print("Clearing existing in-memory laws before loading all from DB.")
            self.lms.clear()
            self.short_names.clear()
            
        for law_record in base_law_records:
            law_db_id, law_code, law_name_from_db, law_regulation_json = law_record
            law_regulation_data = json.loads(law_regulation_json) if isinstance(law_regulation_json, str) else law_regulation_json
            print(f"Processing law from DB: {law_name_from_db} (Code: {law_code}, DB ID: {law_db_id})")

            articles_query = "SELECT data FROM articles WHERE law_id = %s ORDER BY article_number;"
            article_records = fetch_query(self.db_conn, articles_query, (law_db_id,))
            fetched_articles_list = [json.loads(rec[0]) if isinstance(rec[0], str) else rec[0] for rec in article_records or []]

            concepts_query = """
            SELECT DISTINCT lc.data 
            FROM legal_concepts lc 
            LEFT JOIN article_legal_concept alc ON lc.id = alc.legal_concept_id 
            LEFT JOIN articles a ON alc.article_id = a.id 
            WHERE a.law_id = %s OR lc.code LIKE %s; -- Attempt to get all concepts for the law code prefix
            """
            # Using law_name_from_db for the prefix, as it's the name associated with the law in the DB.
            concept_code_prefix_for_like = f"LC_{law_name_from_db}_%"
            concept_records = fetch_query(self.db_conn, concepts_query, (law_db_id, concept_code_prefix_for_like))
            fetched_concepts_list = [json.loads(rec[0]) if isinstance(rec[0], str) else rec[0] for rec in concept_records or []]

            hierarchy_query = "SELECT data FROM law_hierarchy_relationships WHERE main_law_id = %s OR related_law_id = %s;"
            hierarchy_records = fetch_query(self.db_conn, hierarchy_query, (law_db_id, law_db_id))
            fetched_hierarchy_list = [json.loads(rec[0]) if isinstance(rec[0], str) else rec[0] for rec in hierarchy_records or []]

            law_relations_query = "SELECT data FROM law_relationships WHERE main_law_id = %s OR related_law_id = %s OR main_article_id IN (SELECT id FROM articles WHERE law_id = %s) OR related_article_id IN (SELECT id FROM articles WHERE law_id = %s);"
            law_relation_records = fetch_query(self.db_conn, law_relations_query, (law_db_id, law_db_id, law_db_id, law_db_id))
            fetched_law_relations_list = [json.loads(rec[0]) if isinstance(rec[0], str) else rec[0] for rec in law_relation_records or []]

            lm = LawMetadata(
                law_regulation=law_regulation_data, 
                legal_concepts=fetched_concepts_list, 
                hierarchy_relations=fetched_hierarchy_list, 
                law_relations=fetched_law_relations_list, 
                law_articles=fetched_articles_list
            )
            self.lms[lm.law_name] = lm 
            self.short_names[lm.law_name] = lm.law_name 
            print(f"  Reconstructed and added LawMetadata for '{lm.law_name}' to manager.")
            loaded_count += 1
        print(f"Finished loading from DB. {loaded_count} laws processed and added to manager.")

    def export_all_to_json(self):
        """Exports all LawMetadata objects in the manager to JSON files using their short_name as prefix."""
        if not os.path.exists(self.dir_json):
            os.makedirs(self.dir_json)
            print(f"Created directory: {self.dir_json}")
        
        exported_count = 0
        for law_name, lm in self.lms.items():
            safe_short_name = lm.short_name.replace('/', '_').replace('\\', '_')
            file_prefix = os.path.join(self.dir_json, safe_short_name)
            lm.to_json_files(output_prefix=file_prefix)
            exported_count +=1
        print(f"Exported {exported_count} laws to JSON files in '{self.dir_json}' directory.")

    def __repr__(self):
        return f"LawMetadataManager(law_count={len(self.lms)}, db_connected={'Yes' if self.db_conn else 'No'})"




## Interacting with the PostgreSQL Database
This notebook has been enhanced to support PostgreSQL for persistent storage and management of `LawMetadata` objects. This offers several advantages over using only local JSON files, including:
- **Persistent Storage:** Data saved to the database will persist across notebook sessions.
- **Structured Querying:** Although the current analysis tools in this notebook are Python-based, storing data in a relational database allows for powerful SQL querying capabilities externally (e.g., for complex data retrieval, reporting, or integration with other systems).
- **Data Sharing & Centralization:** A database can serve as a central repository for legal metadata, accessible by multiple users or processes (with appropriate database permissions).

**Dual-Mode Capability:**
The notebook, particularly the `LawMetadataMgr` class, is designed to operate in a "dual mode":
1.  **JSON-Only Mode:** If a database connection is not provided to `LawMetadataMgr` during its initialization, it will work exclusively with local JSON files (loading from and saving to the `./json` directory).
2.  **Database-Integrated Mode:** If a database connection object is provided, `LawMetadataMgr` methods like `add_lm` and `remove_lm` will automatically attempt to synchronize changes with the database (i.e., upserting or deleting corresponding records).

This section provides a comprehensive guide to setting up the database, creating the schema, and performing various operations such as loading data from JSON to the DB, retrieving data from the DB into `LawMetadata` objects, deleting data, and using the analysis tools with DB-loaded data.

### A. Database Connection and Core Utility Functions
This cell defines essential functions for database interaction. It's crucial for any subsequent database operations.

**Key Functions Defined Here:**
- `get_db_connection()`: Establishes a connection to the PostgreSQL database.
  - **Configuration Required:** The connection parameters (`DB_NAME`, `DB_USER`, `DB_PASSWORD`, `DB_HOST`, `DB_PORT`) are defined at the beginning of this code cell. 
  - **Best Practice:** Set these using environment variables for security and flexibility (e.g., `os.getenv('PG_DB_USER', 'your_default_user')`). 
  - **Manual Setup:** If not using environment variables, **you must replace the placeholder default values** (like `'your_default_db_name'`) with your actual database credentials and details.
  - **Server Accessibility:** Ensure your PostgreSQL server is running and accessible from the environment where this notebook is executed.
- `execute_query()`: Executes general SQL commands that modify the database (e.g., INSERT, UPDATE, DELETE, DDL like `CREATE TABLE`). It handles transaction commits and rollbacks.
- `fetch_query()`: Executes SELECT queries and fetches results.
- `create_db_schema()`: Reads and executes SQL from `law_db_law.sql` to set up the database tables.
- Helper ID Lookup Functions (e.g., `_get_law_db_id_by_code`): Internal functions used by the main upsert/delete operations to find database primary keys.
- `upsert_law_metadata_to_db()`: A comprehensive function to save or update a `LawMetadata` object and all its constituent parts (regulation, articles, concepts, relations) into the database.
- `delete_law_metadata_from_db()`: Deletes a law and all its associated data from the database based on its unique code.

**Prerequisites for running this cell and subsequent DB operations:**
- `psycopg2-binary` library must be installed (`pip install psycopg2-binary`).
- A PostgreSQL server must be running and accessible.
- Database connection parameters below must be correctly configured.

In [ ]:
# Install psycopg2-binary if you haven't: pip install psycopg2-binary
import psycopg2
import os
import json # Ensure json is imported here for json.dumps

# --- Database Connection Parameters --- 
# IMPORTANT: Configure these for your PostgreSQL instance.
# Using environment variables is recommended for sensitive data like passwords.
# Example: os.getenv('PG_DB_NAME', 'your_default_db_name')
# If not using environment variables, replace the default string values directly.
DB_NAME = os.getenv('PG_DB_NAME', 'your_default_db_name')         # Your database name
DB_USER = os.getenv('PG_DB_USER', 'your_default_user')           # Your PostgreSQL username
DB_PASSWORD = os.getenv('PG_DB_PASSWORD', 'your_default_password') # Your PostgreSQL password
DB_HOST = os.getenv('PG_DB_HOST', 'localhost')                   # Database host (e.g., 'localhost' or an IP address)
DB_PORT = os.getenv('PG_DB_PORT', '5432')                      # Database port (default for PostgreSQL is '5432')
# Ensure the PostgreSQL server is running and accessible with these credentials.

def get_db_connection():
    """Establishes a connection to the PostgreSQL database."""
    conn = None
    try:
        conn = psycopg2.connect(
            dbname=DB_NAME,
            user=DB_USER,
            password=DB_PASSWORD,
            host=DB_HOST,
            port=DB_PORT
        )
        print("Successfully connected to PostgreSQL.")
        return conn
    except psycopg2.Error as e:
        print(f"Error connecting to PostgreSQL: {e}")
        return None

def execute_query(conn, query, params=None):
    """
    Executes a query that modifies the database (INSERT, UPDATE, DELETE).
    Also handles execution of multi-statement SQL scripts.
    
    Args:
        conn: Active database connection.
        query (str): SQL query string or script.
        params (tuple, optional): Parameters for the query. Not typically used for multi-statement scripts.
    """
    cursor = None
    try:
        cursor = conn.cursor()
        cursor.execute(query, params)
        conn.commit()
        print("Query executed successfully.")
    except psycopg2.Error as e:
        if conn:
            conn.rollback()
        print(f"Error executing query: {e}")
        # raise  # Optionally re-raise the exception
    finally:
        if cursor:
            cursor.close()

def fetch_query(conn, query, params=None):
    """
    Executes a query and fetches results (SELECT).
    
    Args:
        conn: Active database connection.
        query (str): SQL query string.
        params (tuple, optional): Parameters for the query. Defaults to None.
        
    Returns:
        list: A list of tuples representing the fetched rows, or None if an error occurs.
    """
    cursor = None
    try:
        cursor = conn.cursor()
        cursor.execute(query, params)
        results = cursor.fetchall()
        return results
    except psycopg2.Error as e:
        print(f"Error fetching query: {e}")
        return None
    finally:
        if cursor:
            cursor.close()

def create_db_schema(conn):
    """
    Reads SQL statements from 'law_db_law.sql' and executes them to create the DB schema.

    Args:
        conn: Active database connection.
    """
    if 1:
        return
    try:
        with open('law_db_law.sql', 'r', encoding='utf-8') as f:
            sql_script = f.read()
        
        print("Attempting to create database schema from law_db_law.sql...")
        execute_query(conn, sql_script) 
        print("Database schema creation process completed.")
        print("Please check for any errors above from execute_query. If 'Query executed successfully.' was printed by execute_query, the script likely ran.")
        
    except FileNotFoundError:
        print("Error: law_db_law.sql not found. Cannot create database schema.")
    except psycopg2.Error as e:
        print(f"Error during schema creation: {e}")
        if conn:
            conn.rollback()
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        if conn:
            conn.rollback()

### --- Helper functions for DB ID lookups --- 
def _get_law_db_id_by_code(law_code, conn):
    """Fetches the database ID of a law by its code."""
    if not law_code: return None
    query = "SELECT id FROM laws WHERE code = %s;"
    result = fetch_query(conn, query, (law_code,))
    return result[0][0] if result else None

def _get_article_db_id_by_law_db_id_and_article_number(law_db_id, article_number, conn):
    """Fetches the database ID of an article by its law_db_id and article_number."""
    if not law_db_id or not article_number: return None
    query = "SELECT id FROM articles WHERE law_id = %s AND article_number = %s;"
    result = fetch_query(conn, query, (law_db_id, article_number))
    return result[0][0] if result else None

def _get_concept_db_id_by_code(concept_code, conn):
    """Fetches the database ID of a legal concept by its code."""
    if not concept_code: return None
    query = "SELECT id FROM legal_concepts WHERE code = %s;"
    result = fetch_query(conn, query, (concept_code,))
    return result[0][0] if result else None

### --- Main DB Upsert/Delete Functions --- 
def delete_law_metadata_from_db(law_code: str, conn):
    """Deletes a law and its related data from the database based on law_code."""
    if not law_code:
        print("Error: Law code cannot be empty for deletion.")
        return False
    print(f"Attempting to delete law with code: {law_code} and its related data...")
    query = "DELETE FROM laws WHERE code = %s;"
    execute_query(conn, query, (law_code,))
    # print(f"Deletion process for law code {law_code} initiated. Check logs from execute_query for status.") # execute_query is verbose enough
    return True

def upsert_law_metadata_to_db(lm: LawMetadata, conn):
    """Upserts a LawMetadata object into the database."""
    law_code = lm.law_regulation.get('代號')
    if not law_code:
        print("Error: Law '代號' (code) is missing from law_regulation. Cannot upsert.")
        return False
    
    print(f"Starting upsert process for law: {lm.law_regulation.get('法規名稱', law_code)} (Code: {law_code})")

    try:
        # Initial Deletion for the main law and its direct dependents via CASCADE
        # print(f"Calling delete_law_metadata_from_db for law code: {law_code}") # upsert implies delete-then-insert
        delete_law_metadata_from_db(law_code, conn) 

        # Insert into laws table
        law_name = lm.law_regulation.get('法規名稱', 'N/A')
        law_data_json = json.dumps(lm.law_regulation)
        law_insert_query = "INSERT INTO laws (code, name, data) VALUES (%s, %s, %s) RETURNING id;"
        law_id_result = fetch_query(conn, law_insert_query, (law_code, law_name, law_data_json))
        
        if not law_id_result or not law_id_result[0]:
            print(f"Error: Failed to insert into 'laws' table for {law_code}. Aborting upsert.")
            conn.rollback()
            return False
        law_db_id = law_id_result[0][0]
        print(f"Inserted into 'laws' table with ID: {law_db_id} for law code {law_code}")

        concept_code_to_db_id_map = {}

        # Insert into articles table
        for article in lm.law_articles:
            article_number = article.get('條號')
            article_data_json = json.dumps(article)
            article_insert_query = "INSERT INTO articles (law_id, article_number, data) VALUES (%s, %s, %s) RETURNING id;"
            article_id_result = fetch_query(conn, article_insert_query, (law_db_id, article_number, article_data_json))
            if article_id_result and article_id_result[0]:
                article['db_id'] = article_id_result[0][0]
            else:
                print(f"  Warning: Failed to insert article {article_number} for law {law_code}.")

        # Insert/Update legal_concepts table
        for concept in lm.legal_concepts:
            concept_code = concept.get('代號')
            concept_name = concept.get('詞彙名稱')
            concept_data_json = json.dumps(concept)
            concept_upsert_query = """
            INSERT INTO legal_concepts (code, name, data) 
            VALUES (%s, %s, %s) 
            ON CONFLICT (code) DO UPDATE SET name = EXCLUDED.name, data = EXCLUDED.data
            RETURNING id;
            """
            concept_id_result = fetch_query(conn, concept_upsert_query, (concept_code, concept_name, concept_data_json))
            if concept_id_result and concept_id_result[0]:
                concept['db_id'] = concept_id_result[0][0]
                concept_code_to_db_id_map[concept_code] = concept['db_id']
            else:
                print(f"  Warning: Failed to upsert legal_concept {concept_code}.")

        # Insert/Update law_hierarchy_relationships table
        for relation in lm.hierarchy_relations:
            rel_code = relation.get('關係代號')
            main_law_code_hier = relation.get('主法規代號')
            related_law_code_hier = relation.get('關聯法規代號')
            hierarchy_type = relation.get('階層關係類型')
            relation_data_json = json.dumps(relation)
            main_law_db_id_hier = _get_law_db_id_by_code(main_law_code_hier, conn)
            related_law_db_id_hier = _get_law_db_id_by_code(related_law_code_hier, conn)
            if main_law_db_id_hier and related_law_db_id_hier and rel_code:
                hier_upsert_query = """
                INSERT INTO law_hierarchy_relationships 
                    (relationship_code, main_law_id, related_law_id, hierarchy_type, data)
                VALUES (%s, %s, %s, %s, %s)
                ON CONFLICT (relationship_code) DO UPDATE SET
                    main_law_id = EXCLUDED.main_law_id,
                    related_law_id = EXCLUDED.related_law_id,
                    hierarchy_type = EXCLUDED.hierarchy_type,
                    data = EXCLUDED.data;
                """
                execute_query(conn, hier_upsert_query, (rel_code, main_law_db_id_hier, related_law_db_id_hier, hierarchy_type, relation_data_json))
            else:
                print(f"  Warning: Skipping law_hierarchy_relationship for {rel_code} due to missing law codes/IDs or relation code.")

        # Insert/Update law_relationships table
        for relation in lm.law_relations:
            rel_code = relation.get('代號')
            main_law_code_rel = relation.get('主法規代號')
            main_article_num_rel = relation.get('主法條條號')
            related_law_code_rel = relation.get('關聯法規代號')
            related_article_num_rel = relation.get('關聯法條條號')
            relationship_type = relation.get('關聯類型', relation.get('relationship_type'))
            relation_data_json = json.dumps(relation)
            main_law_db_id_rel = _get_law_db_id_by_code(main_law_code_rel, conn)
            related_law_db_id_rel = _get_law_db_id_by_code(related_law_code_rel, conn)
            main_article_db_id_rel = None
            if main_law_db_id_rel and main_article_num_rel:
                main_article_db_id_rel = _get_article_db_id_by_law_db_id_and_article_number(main_law_db_id_rel, main_article_num_rel, conn)
            related_article_db_id_rel = None
            if related_law_db_id_rel and related_article_num_rel:
                related_article_db_id_rel = _get_article_db_id_by_law_db_id_and_article_number(related_law_db_id_rel, related_article_num_rel, conn)
            if rel_code and relationship_type:
                law_rel_upsert_query = """
                INSERT INTO law_relationships 
                    (code, relationship_type, main_law_id, main_article_id, related_law_id, related_article_id, data)
                VALUES (%s, %s, %s, %s, %s, %s, %s)
                ON CONFLICT (code) DO UPDATE SET
                    relationship_type = EXCLUDED.relationship_type,
                    main_law_id = EXCLUDED.main_law_id,
                    main_article_id = EXCLUDED.main_article_id,
                    related_law_id = EXCLUDED.related_law_id,
                    related_article_id = EXCLUDED.related_article_id,
                    data = EXCLUDED.data;
                """
                execute_query(conn, law_rel_upsert_query, (
                    rel_code, relationship_type, 
                    main_law_db_id_rel, main_article_db_id_rel,
                    related_law_db_id_rel, related_article_db_id_rel,
                    relation_data_json
                ))
            else:
                print(f"  Warning: Skipping law_relationship for {rel_code} due to missing code or type.")

        # Populate article_legal_concept table
        for article in lm.law_articles:
            article_db_id = article.get('db_id')
            if not article_db_id:
                continue
            related_concept_codes = article.get('相關概念代號列表', [])
            for concept_code in related_concept_codes:
                concept_db_id = concept_code_to_db_id_map.get(concept_code)
                if concept_db_id:
                    alc_insert_query = """
                    INSERT INTO article_legal_concept (article_id, legal_concept_id)
                    VALUES (%s, %s) ON CONFLICT DO NOTHING;
                    """
                    execute_query(conn, alc_insert_query, (article_db_id, concept_db_id))
                else:
                    print(f"    Warning: Concept DB ID not found for code {concept_code} when linking to article {article.get('條號')}.")
        
        conn.commit()
        print(f"Upsert process for law code {law_code} completed successfully.")
        return True
    except Exception as e:
        print(f"An overall error occurred during upsert_law_metadata_to_db for {law_code}: {e}")
        if conn: conn.rollback()
        return False



### B. Creating the Database Schema
This step is crucial and should be performed once to set up the necessary tables in your PostgreSQL database. It executes the SQL commands defined in `law_db_law.sql`.
**Prerequisite:**
- Database connection parameters must be correctly set in the utility cell above.
- The `law_db_law.sql` file must be present in the same directory as this notebook.

In [ ]:
print("--- B. Creating Database Schema ---")
conn_schema = get_db_connection()
if conn_schema:
    try:
        if 0:
            create_db_schema(conn_schema)
    finally:
        conn_schema.close()
        print("DB connection for schema creation closed.")
else:
    print("Failed to connect to the database. Schema creation skipped.")

### C. Loading JSON Data and Upserting to Database
This section demonstrates how to load `LawMetadata` from JSON files and then save it to the database.

#### C.i. Upserting a Single `LawMetadata` Object (from JSON files to DB)
This example loads a specific law (e.g., "政府採購法") from its individual JSON component files, creates a `LawMetadata` object, and then uses the `upsert_law_metadata_to_db` function to save it into the database.
**Prerequisites:**
- Corresponding JSON files (e.g., `json/政府採購法_law_regulation.json`, etc.) must exist.
- Database schema must be created (Section B).

In [ ]:
print("--- C.i. Upserting a Single LawMetadata from JSON to DB ---")
single_law_name_to_load = "政府採購法" 
example_law_code_for_later_use = f"LT_{single_law_name_to_load}" # Default expected code after renew_id
example_law_name_for_later_use = single_law_name_to_load

single_lm_regulation_file = os.path.join(dir_json, f"{single_law_name_to_load}_law_regulation.json")
single_lm_concepts_file = os.path.join(dir_json, f"{single_law_name_to_load}_legal_concepts.json")
single_lm_hierarchy_file = os.path.join(dir_json, f"{single_law_name_to_load}_hierarchy_relations.json")
single_lm_relations_file = os.path.join(dir_json, f"{single_law_name_to_load}_law_relations.json")
single_lm_articles_file = os.path.join(dir_json, f"{single_law_name_to_load}_law_articles.json")

print(f"Attempting to load '{single_law_name_to_load}' from JSON files...")
lm_single_from_json = LawMetadata.from_json_files(
    law_regulation=single_lm_regulation_file,
    legal_concepts=single_lm_concepts_file,
    hierarchy_relations=single_lm_hierarchy_file,
    law_relations=single_lm_relations_file,
    law_articles=single_lm_articles_file
)

if lm_single_from_json:
    print(f"Successfully loaded '{lm_single_from_json.law_name}' from JSON files.")
    # lm_single_from_json.renew_id() # Call if IDs in JSON are not standardized for DB
    # example_law_code_for_later_use = lm_single_from_json.law_regulation.get('代號', example_law_code_for_later_use)
    # print(f"Law code to be used: {example_law_code_for_later_use}")

    conn_single_upsert = get_db_connection()
    if conn_single_upsert:
        try:
            print(f"Attempting to upsert '{lm_single_from_json.law_name}' (Code: {lm_single_from_json.law_regulation.get('代號')}) to database...")
            success = upsert_law_metadata_to_db(lm_single_from_json, conn_single_upsert)
            print(f"Upsert of '{lm_single_from_json.law_name}' to DB reported success: {success}")
            if success:
                 # Update based on what was actually upserted, especially if renew_id was called.
                 example_law_code_for_later_use = lm_single_from_json.law_regulation.get('代號')
                 example_law_name_for_later_use = lm_single_from_json.law_name
                 print(f"Stored for later use: Code='{example_law_code_for_later_use}', Name='{example_law_name_for_later_use}'")
        finally:
            conn_single_upsert.close()
            print("DB connection for single upsert closed.")
    else:
        print("Failed to connect to DB for single law upsert.")
else:
    print(f"Failed to load '{single_law_name_to_load}' from JSON files. Ensure all component files exist in '{dir_json}'.")

#### C.ii. Upserting Multiple `LawMetadata` Objects via `LawMetadataMgr` (from JSON files to DB)
This demonstrates using `LawMetadataMgr` to load multiple laws from JSON files. If the manager is initialized with a database connection, laws are automatically upserted to the DB when added to the manager via `add_lm` (which is called by `load_lm_bynames_from_json`).
**Prerequisites:**
- JSON files for the specified laws (e.g., "政府採購法", "民法") must exist.
- Database schema must be created.

In [ ]:
print("--- C.ii. Upserting Multiple Laws via LawMetadataMgr (JSON to DB) ---")
 
available_json_laws = get_law_names_from_directory(dir_json)
print(f"Available JSON laws in '{dir_json}': {available_json_laws}")

if 0:
    law_short_names_for_mgr_upsert = ["政府採購法", "民法"]
else:
    law_short_names_for_mgr_upsert = available_json_laws 

actual_laws_to_process_mgr = [name for name in law_short_names_for_mgr_upsert if name in available_json_laws]

if not actual_laws_to_process_mgr:
    print(f"None of the specified laws ({law_short_names_for_mgr_upsert}) found as JSON files in '{dir_json}'. Skipping multi-upsert demo.")
else:
    print(f"Will attempt to load and upsert: {actual_laws_to_process_mgr}")
    conn_mgr_upsert = get_db_connection()
    if conn_mgr_upsert:
        lmmgr_for_multi_upsert = LawMetadataMgr(db_conn=conn_mgr_upsert)
        try:
            # load_lm_bynames_from_json will call add_lm, which in turn calls upsert_law_metadata_to_db
            lmmgr_for_multi_upsert.load_lm_bynames_from_json(actual_laws_to_process_mgr)
            print(f"Finished processing {len(actual_laws_to_process_mgr)} laws for upsert via LawMetadataMgr.")
            print(f"Laws in manager: {list(lmmgr_for_multi_upsert.lms.keys())}")
        except Exception as e_mgr_upsert:
            print(f"An error occurred during multi-upsert via LawMetadataMgr: {e_mgr_upsert}")
        finally:
            conn_mgr_upsert.close()
            print("DB connection for multi-upsert via LawMetadataMgr closed.")
    else:
        print("Failed to connect to DB for multi-law upsert via LawMetadataMgr.")

### D. Loading `LawMetadata` from Database using `LawMetadataMgr`
These examples show how to use `LawMetadataMgr` (initialized with a database connection) to load `LawMetadata` objects directly from the PostgreSQL database. When `LawMetadataMgr` is initialized with a `db_conn`, its methods `add_lm` and `remove_lm` interact with the database. The `load_lm_from_db` method specifically fetches data from the database to populate the manager's in-memory collection.
**Prerequisites:**
- Database schema must be created (Section B).
- Data for the laws to be loaded must exist in the database (e.g., from Section C).

In [26]:
print("--- D.i. Loading a Specific Law from DB into LawMetadataMgr ---")
conn_load_single_db = get_db_connection()
if conn_load_single_db:
    lmmgr_load_single = LawMetadataMgr(db_conn=conn_load_single_db)
    try:
        law_code_to_load_specific = example_law_code_for_later_use if 'example_law_code_for_later_use' in locals() and example_law_code_for_later_use else "LT_政府採購法"
        law_name_to_find_specific = example_law_name_for_later_use if 'example_law_name_for_later_use' in locals() and example_law_name_for_later_use else "政府採購法"

        print(f"Attempting to load law with code '{law_code_to_load_specific}' from DB...")
        lmmgr_load_single.load_lm_from_db(law_code_to_load=law_code_to_load_specific)
        
        loaded_lm_specific = lmmgr_load_single.find_lm(law_name_to_find_specific)
        if loaded_lm_specific:
            print(f"Successfully loaded and found '{loaded_lm_specific.law_name}' (Code: {loaded_lm_specific.law_regulation.get('代號')}) from DB.")
            print(f"  Number of articles: {len(loaded_lm_specific.law_articles)}")
            print(f"  Number of concepts: {len(loaded_lm_specific.legal_concepts)}")
        else:
            print(f"Could not find law with name '{law_name_to_find_specific}' (tried code '{law_code_to_load_specific}') in manager after DB load attempt.")
    finally:
        conn_load_single_db.close()
        print("DB connection for loading single law closed.")
else:
    print("Failed to connect to DB for loading a single law.")

--- D.i. Loading a Specific Law from DB into LawMetadataMgr ---
Successfully connected to PostgreSQL.
LawMetadataMgr initialized with a database connection.
Attempting to load law with code 'LT_政府採購法' from DB...
No laws found in DB for code 'LT_政府採購法'.
Could not find law with name '政府採購法' (tried code 'LT_政府採購法') in manager after DB load attempt.
DB connection for loading single law closed.


In [ ]:
print("\n--- D.ii. Loading All Laws from DB into LawMetadataMgr ---")
conn_load_all_db = get_db_connection()
if conn_load_all_db:
    lmmgr_load_all = LawMetadataMgr(db_conn=conn_load_all_db)
    try:
        print("Attempting to load ALL laws from DB...")
        lmmgr_load_all.load_lm_from_db() # Load all
        print(f"Total laws in manager after loading all from DB: {len(lmmgr_load_all.lms)}")
        if lmmgr_load_all.lms:
            print("Listing loaded laws:")
            for law_obj_name in lmmgr_load_all.lms.keys():
                print(f"- {law_obj_name}")
        else:
            print("No laws were loaded from the database.")
    finally:
        conn_load_all_db.close()
        print("DB connection for loading all laws closed.")
else:
    print("Failed to connect to DB for loading all laws.")

### E. Deleting `LawMetadata` from Database
This demonstrates how to delete a specific law and its associated data from the database using the standalone `delete_law_metadata_from_db` function. Deletion can also occur via `LawMetadataMgr.remove_lm()` if the manager is DB-connected.
**Prerequisites:**
- The law to be deleted must exist in the database.
- The `law_code` for the law must be known (e.g., from a previous upsert operation).

In [ ]:
print("--- E. Deleting a Specific Law from DB ---")
law_code_to_delete_example = example_law_code_for_later_use if 'example_law_code_for_later_use' in locals() and example_law_code_for_later_use else "LT_政府採購法"
law_name_to_verify_delete = example_law_name_for_later_use if 'example_law_name_for_later_use' in locals() and example_law_name_for_later_use else "政府採購法"

print(f"Attempting to delete law with code: '{law_code_to_delete_example}'")

conn_delete = get_db_connection()
if conn_delete:
    try:
        # 1. Verify it exists (optional, but good for demo)
        lmmgr_verify_b_delete = LawMetadataMgr(db_conn=conn_delete)
        lmmgr_verify_b_delete.load_lm_from_db(law_code_to_load=law_code_to_delete_example)
        if lmmgr_verify_b_delete.find_lm(law_name_to_verify_delete):
            print(f"Law '{law_name_to_verify_delete}' (Code: {law_code_to_delete_example}) found in DB before deletion.")
        else:
            print(f"Warning: Law '{law_name_to_verify_delete}' (Code: {law_code_to_delete_example}) NOT found in DB before attempting deletion.")

        # 2. Perform deletion using the direct function
        print(f"Calling delete_law_metadata_from_db for code '{law_code_to_delete_example}'...")
        delete_success = delete_law_metadata_from_db(law_code_to_delete_example, conn_delete)
        print(f"Direct delete_law_metadata_from_db reported: {'Success (likely)' if delete_success else 'Attempted, check logs'}")

        # 3. Verify it's deleted
        print(f"Verifying deletion of '{law_name_to_verify_delete}' (Code: {law_code_to_delete_example})...")
        lmmgr_verify_a_delete = LawMetadataMgr(db_conn=conn_delete) 
        lmmgr_verify_a_delete.load_lm_from_db(law_code_to_load=law_code_to_delete_example)
        if not lmmgr_verify_a_delete.find_lm(law_name_to_verify_delete):
            print(f"Successfully verified: Law '{law_name_to_verify_delete}' (Code: {law_code_to_delete_example}) is no longer found in DB.")
        else:
            print(f"Error: Law '{law_name_to_verify_delete}' (Code: {law_code_to_delete_example}) still found in DB after deletion attempt.")
    
    except Exception as e_del:
        print(f"An error occurred during the deletion demonstration: {e_del}")
    finally:
        conn_delete.close()
        print("DB connection for deletion example closed.")
else:
    print("Failed to connect to DB for deletion example.")

# 進階搜尋與分析 (Analysis Tools)
This section defines and demonstrates various analysis tools that can operate on `LawMetadata` objects, whether they are loaded from JSON files or from the database.

In [ ]:
# Analysis function definitions (keyword_search, category_filter, etc.)
def keyword_search(metadata, keyword, fields_to_search):
    """
    在指定的 MetaData 欄位中搜尋關鍵字。
    Args:
        metadata (LawMetaData): LawMetaData 物件。
        keyword (str): 要搜尋的關鍵字。
        fields_to_search (list of str): 要搜尋的欄位名稱列表 (例如 ['簡述', '定義'])。
    Returns:
        list: 包含符合條件的 MetaData 物件 (或其子物件)。
    """
    results = []
    if not metadata: return results # Guard against None metadata
    if "簡述" in fields_to_search and hasattr(metadata, 'law_regulation') and metadata.law_regulation:
        if keyword.lower() in metadata.law_regulation.get("簡述", "").lower():
            results.append(metadata.law_regulation)
    if "定義" in fields_to_search and hasattr(metadata, 'legal_concepts') and metadata.legal_concepts:
        for concept in metadata.legal_concepts:
            if keyword.lower() in concept.get("定義", "").lower():
                results.append(concept)
    return results

def category_filter(metadata, category_type, metadata_type="hierarchy_relations"):
    """
    篩選特定類型的 MetaData。
    Args:
        metadata (LawMetaData): LawMetaData 物件。
        category_type (str or Enum): 要篩選的類別/類型 (例如 "子法規", ConceptCategory.CORE_CONCEPT_DEFINITION)。
        metadata_type (str, optional): 要篩選的 MetaData 類型 (預設為 "hierarchy_relations")。
    Returns:
        list: 符合類別/類型的 MetaData 物件列表。
    """
    results = []
    if not metadata: return results # Guard against None metadata
    data_list = getattr(metadata, metadata_type, []) 
    for item in data_list:
        if item.get("階層關係類型") == category_type or \
           (isinstance(category_type, Enum) and item.get("概念類別") == category_type.value) or \
           (not isinstance(category_type, Enum) and item.get("概念類別") == category_type):
            results.append(item)
    return results

def combined_search_hierarchy(metadata, category_type, related_law_keyword):
    """
    複合條件搜尋法規階層關係 (特定類型 + 關聯法規關鍵字)。
    Args:
        metadata (LawMetaData): LawMetaData 物件。
        category_type (str): 要篩選的階層關係類型 (例如 "子法規")。
        related_law_keyword (str): 關聯法規名稱中要包含的關鍵字 (例如 "施行細則")。
    Returns:
        list: 符合複合條件的法規階層關係 MetaData 物件列表。
    """
    results = []
    if not metadata: return results # Guard against None metadata
    for relation in metadata.hierarchy_relations:
        if relation.get("階層關係類型") == category_type and related_law_keyword.lower() in relation.get("關聯法規", "").lower():
            results.append(relation)
    return results

def generate_mermaid_diagram(metadata):
    """
    生成法規關係圖的 mermaid 語法。
    Args:
        metadata (LawMetaData): LawMetaData 物件。
    Returns:
        str: mermaid 語法字串。
    """
    mermaid_lines = []
    if not metadata: return "graph TD\n    %% No metadata provided"
    for relation in metadata.hierarchy_relations:
        main_law = relation.get("主法規", "N/A")
        related_law = relation.get("關聯法規", "N/A")
        relation_type = relation.get("階層關係類型", "related")
        mermaid_lines.append(f'    {main_law} -->|{relation_type}| {related_law}')
    if not mermaid_lines:
        return "graph TD\n    %% No hierarchy relations found"
    return "graph TD\n" + "\n".join(mermaid_lines)

def generate_article_mermaid_diagram(metadata):
    """
    生成法條關係圖的 mermaid 語法。
    Args:
        metadata (LawMetaData): LawMetaData 物件。
    Returns:
        str: mermaid 語法字串。
    """
    mermaid_lines = ["graph TD"]
    if not metadata: return "graph TD\n    %% No metadata provided"
    for article in metadata.law_articles:
        article_id = article.get("條號", "UnknownArticle")
        related_articles_map = article.get("法條關聯性", {}) 
        for rel_type, related_items in related_articles_map.items():
            if related_items and isinstance(related_items, list):
                for v_item in related_items:
                    v_clean = str(v_item).replace("、","_").replace("，","_").replace(" ","_").replace("『","_").replace("』","_")
                    mermaid_lines.append(f'    A_{article_id}["{article_id}"] --> |{rel_type}| R_{v_clean}["{v_item}"]')
    if len(mermaid_lines) == 1:
        return "graph TD\n    %% No article relations found"
    return "\n".join(mermaid_lines)

print("Analysis tool functions defined.")

### F. Analyzing DB-Loaded Data
This section demonstrates using the analysis tools defined above with `LawMetadata` objects loaded from the PostgreSQL database.
**Prerequisite:** Ensure that a law (e.g., "政府採購法" with its corresponding code) has been successfully upserted into the database.

In [ ]:
print("--- F. Testing Analysis Tools with DB-Loaded LawMetadata ---")
conn_analysis_db = get_db_connection()
lm_db_for_analysis = None

if conn_analysis_db:
    try:
        lmmgr_db_analysis_tools = LawMetadataMgr(db_conn=conn_analysis_db)
        
        law_code_for_analysis = example_law_code_for_later_use if 'example_law_code_for_later_use' in locals() and example_law_code_for_later_use else "LT_政府採購法"
        law_name_for_analysis = example_law_name_for_later_use if 'example_law_name_for_later_use' in locals() and example_law_name_for_later_use else "政府採購法"
        
        print(f"Attempting to load '{law_name_for_analysis}' (code: {law_code_for_analysis}) from DB for analysis...")
        lmmgr_db_analysis_tools.load_lm_from_db(law_code_to_load=law_code_for_analysis)
        lm_db_for_analysis = lmmgr_db_analysis_tools.find_lm(law_name_for_analysis)

        if lm_db_for_analysis:
            print(f"\nSuccessfully loaded '{law_name_for_analysis}' from DB for analysis.")
            
            print(f"\n--- Keyword Search on DB-loaded '{law_name_for_analysis}' (Keyword: '採購') ---")
            db_keyword_results = keyword_search(lm_db_for_analysis, "採購", ["簡述", "定義"])
            if db_keyword_results:
                for res in db_keyword_results:
                    print(f"  - Found in: {res.get('詞彙名稱', res.get('法規名稱', 'N/A'))}, Content: {res.get('定義', res.get('簡述', ''))[:100]}...")
            else:
                print("  No results for keyword search.")

            print(f"\n--- Category Filter on DB-loaded '{law_name_for_analysis}' (Category: '子法規') ---")
            db_category_results = category_filter(lm_db_for_analysis, "子法規")
            if db_category_results:
                for res in db_category_results:
                    print(f"  - Main: {res.get('主法規')}, Related: {res.get('關聯法規')}, Type: {res.get('階層關係類型')}")
            else:
                print("  No results for category filter '子法規'.")
            
            print(f"\n--- Mermaid Diagram for DB-loaded '{law_name_for_analysis}' (Hierarchy) ---")
            db_mermaid_output = generate_mermaid_diagram(lm_db_for_analysis)
            print(db_mermaid_output)
        else:
            print(f"Failed to load or find '{law_name_for_analysis}' from DB for analysis. Ensure it was upserted with code '{law_code_for_analysis}'.")

    except Exception as e_analysis_db:
        print(f"An error occurred during DB analysis demonstration: {e_analysis_db}")
    finally:
        if conn_analysis_db and not conn_analysis_db.closed:
            conn_analysis_db.close()
            print("\nDB connection for analysis closed.")
else:
    print("Failed to connect to DB for analysis demonstration.")


In [ ]:
if 0: # Original examples with JSON-loaded data (lmmgr_json_ops)
    print("\n--- Testing Analysis Tools with JSON-Loaded LawMetadata (from lmmgr_json_ops) ---")
    # This cell is for comparison, using LawMetadata loaded from JSON files via lmmgr_json_ops.
    # Ensure lmmgr_json_ops is initialized and populated (e.g., from cell under '法規管理使用' section if it was run).
    if 'lmmgr_json_ops' in locals() and lmmgr_json_ops.lms and len(lmmgr_json_ops.lms) > 0:
        first_json_law_name = list(lmmgr_json_ops.lms.keys())[0]
        lm_json_test = lmmgr_json_ops.find_lm(first_json_law_name)
        
        if lm_json_test:
            print(f"Using '{lm_json_test.law_name}' loaded from JSON for comparison.")
            print(f"\n--- Keyword Search on JSON-loaded '{lm_json_test.law_name}' (Keyword: '公開') ---")
            json_keyword_results = keyword_search(lm_json_test, "公開", ["簡述", "定義"])
            if json_keyword_results:
                for res in json_keyword_results:
                    print(f"  - Found in: {res.get('詞彙名稱', res.get('法規名稱', 'N/A'))}, Content: {res.get('定義', res.get('簡述', ''))[:100]}...")
            else:
                print("  No results for keyword search.")

            print(f"\n--- Category Filter on JSON-loaded '{lm_json_test.law_name}' (Category: '子法規') ---")
            json_category_results = category_filter(lm_json_test, "子法規")
            if json_category_results:
                for res in json_category_results:
                    print(f"  - Main: {res.get('主法規')}, Related: {res.get('關聯法規')}, Type: {res.get('階層關係類型')}")
            else:
                print("  No results for category filter '子法規'.")
            
            print(f"\n--- Mermaid Diagram for JSON-loaded '{lm_json_test.law_name}' (Hierarchy) ---")
            json_mermaid_output = generate_mermaid_diagram(lm_json_test)
            print(json_mermaid_output)
        else:
            print(f"Could not find law '{first_json_law_name}' in lmmgr_json_ops to run JSON-based analysis tests.")
    else:
        print("LawMetadata manager for JSON (lmmgr_json_ops) not populated or empty. Skipping JSON-based analysis tests.")

# 單法規使用

In [ ]:
law_name = "政府採購法施行細則" # Or dynamically extract from law_regulation.json if needed

filepaths = {
    "law_regulation": f"{dir_json}/{law_name}_law_regulation.json",
    "legal_concepts": f"{dir_json}/{law_name}_legal_concepts.json",
    "hierarchy_relations": f"{dir_json}/{law_name}_hierarchy_relations.json",
    "law_relations": f"{dir_json}/{law_name}_law_relations.json",
    "law_articles": f"{dir_json}/{law_name}_law_articles.json"
}


# 1. Read from separate JSON files and create LawMetadata object
lm = LawMetadata.from_json_files(**filepaths)


if lm:
    print(lm)

    # 2. Access Metadata content (example)
    print("\nLaw Regulation Name:", lm.law_regulation.get("法規名稱"))
    print("\nFirst Legal Concept Name:", lm.legal_concepts[0].get("詞彙名稱"))

    if 0:
        lm.renew_id()
    # 3. Modify Metadata content (example)
    if 0:
        lm.law_regulation["版本"] = "20250312-Test Version"
        new_concept = {
            "代號": "concept-gpa-new-concept",
            "詞彙名稱": "New Concept",
            "定義": "This is a new concept definition.",
            "相關概念": [],
            "相關法條": [],
            "概念類別": "新增概念",
            "同義詞": [],
            "台灣觀點": "Taiwan Viewpoint.",
            "範例": "Example here.",
            "語意向量": "[]"
        }
        lm.legal_concepts.append(new_concept)
if 0:
    # 4. Export LawMetadata object to separate JSON files
    lm.to_json_files(output_prefix=f"{dir_json}/{law_name}_M") # Exports to gpa_modified_*.json
if 1:
    conn = get_db_connection()
    upsert_law_metadata_to_db(lm,conn)

# 法規管理使用

In [ ]:
if 0: #法律管理
    lmmgr = LawMetadataMgr()
    law_names = get_law_names_from_directory("./json")
    #print(law_names)
    #lmmgr.load_lm_bynames(["憲法合併","憲法增修合併","刑法","民法","行政程序法","預算法","政府採購法","政府採購法施行細則"])
    lmmgr.load_lm_bynames_from_json(law_names)   
    lms = lmmgr.lms
if 0: #法律列表
    for law_name in lms.keys():
        #print(f"{lms[law_name].law_name}-{lms[law_name].short_name}" )
        print(f"{lms[law_name].law_name}" )
if 0: #法律概念列表
    for law_name in lms.keys():
        print(f"----- {law_name} -----")
        for lc in lms[law_name].legal_concepts:
            print(f"{lc['詞彙名稱']}")
    #print(lms['中華民國憲法(合併增修條文)'].legal_concepts)
if 0: # 某法跟什麼法有關係如 公司法
    target = "公司法"
    for law_name in lms.keys():
        if target in lms[law_name].law_regulation['相關法規']:
            print(f"{law_name} 跟 {target} 有關係")
if 1: # 某個關鍵字的總覽
    pass
 

## LLM 建構 json
- 由於 LLM 的產出有時遇到小問題，所以使用時暫時需開開關關

In [ ]:
import base64
import os
# from google import genai # Assuming genai might not be available in all test environments
# from google.genai import types
import time

def generate(client,files,law_name,user_prompt,file_path):
    # This function is a placeholder if genai is not available.
    # To use it, ensure google.genai is installed and configured.
    print(f"[INFO] Generate function called for {law_name} with prompt: {user_prompt[:50]}...")
    print(f"[INFO] Output would be written to {file_path}")
    # Actual generation logic commented out for environments without genai
    """
    model = "gemini-2.0-flash-thinking-exp-01-21"
    if 1:
        contents = [
            types.Content(
                role="user",
                parts=[
                    types.Part.from_uri(
                        file_uri=files[0].uri,
                        mime_type=files[0].mime_type,
                    ),
                    types.Part.from_uri(
                        file_uri=files[1].uri,
                        mime_type=files[1].mime_type,
                    ),
                    types.Part.from_text(text=user_prompt),
                ],
            )
        ]
        generate_content_config = types.GenerateContentConfig(
            temperature=0.7,
            top_p=0.95,
            top_k=64,
            max_output_tokens=65536,
            response_mime_type="text/plain",
            system_instruction=[
                types.Part.from_text(text="""請以台灣人的立場，用繁體中文回答"""),
            ],
        )
        
        print(f"Q::{user_prompt}")

        response_text = ""
        for chunk in client.models.generate_content_stream(
            model=model,
            contents=contents,
            config=generate_content_config,
        ):
            response_text += chunk.text
            #print(chunk.text, end="")
        print(f"A::{response_text}")

        try:
            with open(file_path, 'w', encoding='utf-8') as file:
                file.write(f"Q::{user_prompt}\n")
                file.write(f"A::{response_text}")
                print(f"Content written to {file_path}")
        except Exception as e:
            print(f"An error occurred: {e}")
    """
    pass # Placeholder if genai is not used

api_key=os.environ.get("GEMINI_API_KEY")
client = None # Placeholder for genai.Client
files_for_llm = [] # Placeholder for uploaded files

law_name="憲法增修合併" # Example law name for LLM section

if 0: # This entire LLM section is turned off by default for this refactoring step
    # Ensure google.genai is imported and client is initialized if this block is enabled
    # client = genai.Client(api_key=api_key)
    # files_for_llm = [
    #     client.files.upload(file=f"{law_name}.md",config={'mime_type':"text/markdown"}),
    #     client.files.upload(file="法律語法形式化.md",config={'mime_type':"text/markdown"}),
    # ]
    pass

if 0: # 產生非法條的 Meta data (Turned off)
    prompt_list=[
        ['law_regulation',f"""根據{law_name}的整體資訊，按照法律語法形式化的設計，依照裡面範例格式，產生法規 Meta Data,盡可能詳列資訊，不要省略"""],
        ['legal_concepts',f"""根據{law_name}的整體資訊，按照法律語法形式化的設計，依照裡面範例格式，產生法律概念 Meta Data (Legal Concept Meta Data)，注意並非 法規 Meta Data，請列出全部概念，不要省略"""],
        ['hierarchy_relations',f"""根據{law_name}的整體資訊，按照法律語法形式化的設計，依照裡面範例格式，產生法規階層關係 Meta Data，不包含法條間關聯性，請列出全部法規間階層關係，尤其包含上位關係，不要省略"""],
        ['law_relations',f"""根據{law_name}的整體資訊，按照法律語法形式化的設計，依照裡面範例格式，產生法規關聯性 Meta Data，不包含本法規內部法條之間的關聯性，請列出法規間全部關聯性，不要省略"""]
    ]
    for prompt_pair in prompt_list:
        file_path_txt = f"{dir_txt}/{law_name}_{prompt_pair[0]}.txt"
        if client and files_for_llm: generate(client,files_for_llm,law_name,prompt_pair[1],file_path_txt)
        
        file_path_json = f"{dir_json}/{law_name}_{prompt_pair[0]}.json"
        if os.path.exists(file_path_txt):
            regex = r"```(?:json)*\n(.*)```"
            lines = handle_regex(regex,file_path_txt,"col1")
            if lines:
                json_string = "\n".join(lines)
                json_string = re.sub(r'//.*', '', json_string)
                json_string = re.sub(r'# .*', '', json_string)
                try:
                    json_object = json.loads(json_string)
                    print(json_object)
                    with open(file_path_json, 'w', encoding='utf-8') as f:
                        json.dump(json_object, f, indent=2, ensure_ascii=False)
                except json.JSONDecodeError as je:
                    print(f"JSON Decode Error for {file_path_txt}: {je}")
            else:
                print(f"No JSON content found in {file_path_txt}")

if 0: # 取得最大條號 ，也可手動填入結果 (Turned off)
    # ... (LLM code remains off for this refactoring)
    pass
    
if 0: # 產生法條 Meta Data (Turned off)
    # ... (LLM code remains off for this refactoring)
    pass

if 0: #從檔案內組合法條 Meta data (Turned off)
    # ... (LLM code remains off for this refactoring)
    pass